In [1]:
import pandas as pd 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt

In [65]:
newsgroups = fetch_20newsgroups(subset='train')

In [3]:
newsgroups.keys()

['description', 'DESCR', 'filenames', 'target_names', 'data', 'target']

In [4]:
df = pd.DataFrame(newsgroups['data'], columns=['text'])
print df.iloc[0,0]

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [5]:
df['text_lower'] = df['text'].str.lower()

In [6]:
df['text_lower'].head(2)

0    from: lerxst@wam.umd.edu (where's my thing)\ns...
1    from: guykuo@carson.u.washington.edu (guy kuo)...
Name: text_lower, dtype: object

In [ ]:
CountVectorizer?

In [7]:
count_vectorizer = CountVectorizer(stop_words='english',
                                  max_features=10000)

In [8]:
# fit will just give instructions, adding transform puts it into our df 

corpus_1 = count_vectorizer.fit_transform(df['text_lower'].values)

In [9]:
# heres our sparse matrix

corpus_1

<11314x10000 sparse matrix of type '<type 'numpy.int64'>'
	with 1004156 stored elements in Compressed Sparse Row format>

In [ ]:
TruncatedSVD?

In [10]:
tsvd = TruncatedSVD()

In [11]:
tsvd_word_data = tsvd.fit_transform(corpus_1)

In [12]:
tsvd_word_data[0:5]

array([[  1.17834553e-03,   6.35334200e+00],
       [  1.52753858e-03,   1.84488757e-01],
       [  1.58567063e-03,   3.39016519e-01],
       [  2.44659360e-03,   2.73578555e-01],
       [  1.96677117e-03,   2.66978941e-01]])

In [13]:
# transformed into two columns (made new features)
# all data and text from original df has been transformed into two columnns

tsvd_word_data.shape

(11314, 2)

looking at all 10000 words we gave it and made two columns. lets check it before moving on the kmeans

In [14]:
newsgroups['target_names']

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [15]:
targets = pd.DataFrame(newsgroups['target'], columns=['target_names'])
targets.head()
targets['target_names'].value_counts()

10    600
15    599
8     598
9     597
11    595
13    594
7     594
14    593
5     593
12    591
2     591
3     590
6     585
1     584
4     578
17    564
16    546
0     480
18    465
19    377
Name: target_names, dtype: int64

In [16]:
# looking to see which group to assign to 1's

zip(range(0,21), newsgroups['target_names'])

[(0, 'alt.atheism'),
 (1, 'comp.graphics'),
 (2, 'comp.os.ms-windows.misc'),
 (3, 'comp.sys.ibm.pc.hardware'),
 (4, 'comp.sys.mac.hardware'),
 (5, 'comp.windows.x'),
 (6, 'misc.forsale'),
 (7, 'rec.autos'),
 (8, 'rec.motorcycles'),
 (9, 'rec.sport.baseball'),
 (10, 'rec.sport.hockey'),
 (11, 'sci.crypt'),
 (12, 'sci.electronics'),
 (13, 'sci.med'),
 (14, 'sci.space'),
 (15, 'soc.religion.christian'),
 (16, 'talk.politics.guns'),
 (17, 'talk.politics.mideast'),
 (18, 'talk.politics.misc'),
 (19, 'talk.religion.misc')]

In [17]:
y = targets['target_names'].apply(lambda x: 1 if x == 16 else 0)
y.head()
y.dtypes

dtype('int64')

In [18]:
y[(y ==1)].head()

5      1
39     1
67     1
81     1
127    1
Name: target_names, dtype: int64

In [ ]:
# check to make sure we are talking about the topic we are looking for

# print df.iloc[39,0]

In [19]:
y

0        0
1        0
2        0
3        0
4        0
5        1
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
11284    0
11285    0
11286    0
11287    0
11288    0
11289    0
11290    0
11291    0
11292    0
11293    0
11294    0
11295    0
11296    0
11297    0
11298    0
11299    0
11300    0
11301    0
11302    0
11303    0
11304    0
11305    0
11306    0
11307    0
11308    0
11309    0
11310    0
11311    0
11312    0
11313    0
Name: target_names, dtype: int64

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split    

In [21]:
def tts_and_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    rf = RandomForestClassifier()
    rf.fit(x_train, y_train)
    print rf.score(x_test, y_test)

In [22]:
tsvd_word_data

array([[  1.17834553e-03,   6.35334200e+00],
       [  1.52753858e-03,   1.84488757e-01],
       [  1.58567063e-03,   3.39016519e-01],
       ..., 
       [  4.77910886e-04,   1.14773774e-01],
       [  1.36892826e-03,   2.30511523e-01],
       [  1.74614197e-03,   1.97056300e-01]])

In [23]:
tts_and_model(tsvd_word_data, y.values)

0.947864506627


In [24]:
tsvd_200 = TruncatedSVD(n_components=200)
tsvd_200 = tsvd_200.fit_transform(corpus_1)

In [25]:
tts_and_model(tsvd_200, y)

0.960235640648


In [26]:
ele_y = targets['target_names'].apply(lambda x: 1 if x == 12 else 0)
ele_y.head()

0    0
1    0
2    0
3    0
4    0
Name: target_names, dtype: int64

In [27]:
print tts_and_model(tsvd_word_data, ele_y)
print tts_and_model(tsvd_200, ele_y)

0.946980854197
None
0.950515463918
None


In [28]:
print tts_and_model(tsvd_word_data,  targets['target_names'])
print tts_and_model(tsvd_200,  targets['target_names'])

0.0695139911635
None
0.258615611193
None


lets create a kmeans model

In [29]:
km = KMeans(n_clusters=2)

In [30]:
tsvd_word_data_stand = StandardScaler().fit_transform(tsvd_word_data)

In [31]:
km.fit(tsvd_word_data_stand)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=2, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [32]:
pd.Series(km.labels_).value_counts()

0    11303
1       11
dtype: int64

In [ ]:
plt.scatter(tsvd_word_data[:,0], tsvd_word_data[:,1], c=km.labels_, cmap='viridis')
plt.xlabel('TSVD Component 1')
plt.ylabel('TSVD Component 2')
plt.show()

In [45]:
def kmeans_graph(x,k):
    km = KMeans(n_clusters=k)
    km.fit(x)
    plt.scatter(x[:,0], x[:,1], c=km.labels_, cmap='viridis')
    plt.xlabel('TSVD Component 1')
    plt.ylabel('TSVD Component 2')
    plt.title('Scatter plot of KMeands, %s Clusters' % k)
    plt.show()
    

In [46]:
def silhouette_selection(x, k):
    scores = []
    for iteration in range(10,k):
        print 'I am on iteration %s' % iteration
        km = KMeans(n_clusters=iteration)
        km.fit(x)
        scores.append((iteration, metrics.silhouette_score(x, km.labels_)))
    ks = [k[0] for k in scores]
    sils = [k[1] for k in scores]
    plt.plot(ks, sils)
    plt.ylabel('Silhouette Score')
    plt.xlabel('K in KMeans')
    plt.show
    print scores

In [34]:
# will need to re-run this to show graph

silhouette_selection(tsvd_word_data, 26)

I am on iteration 2
I am on iteration 3
I am on iteration 4
I am on iteration 5
I am on iteration 6
I am on iteration 7
I am on iteration 8
I am on iteration 9
I am on iteration 10
I am on iteration 11
I am on iteration 12
I am on iteration 13
I am on iteration 14
I am on iteration 15
I am on iteration 16
I am on iteration 17
I am on iteration 18
I am on iteration 19
I am on iteration 20
I am on iteration 21
I am on iteration 22
I am on iteration 23
I am on iteration 24
I am on iteration 25


In [37]:
count_vectorizer_all = CountVectorizer(stop_words='english')
corpus_2 = count_vectorizer_all.fit_transform(df['text_lower'].values)

In [39]:
corpus_2

<11314x129797 sparse matrix of type '<type 'numpy.int64'>'
	with 1300761 stored elements in Compressed Sparse Row format>

In [41]:
tv = TruncatedSVD()
tsvd_all = tv.fit_transform(corpus_2)

In [43]:
tsvd_all[0:5]

array([[ 0.00117836,  0.14736386],
       [ 0.00152756,  0.18883465],
       [ 0.00158569,  0.30486347],
       [ 0.00244661,  0.2542514 ],
       [ 0.00196696,  0.25336539]])

In [44]:
silhouette_selection(tsvd_all, 26)

I am on iteration 10
I am on iteration 11
I am on iteration 12
I am on iteration 13
I am on iteration 14
I am on iteration 15
I am on iteration 16
I am on iteration 17
I am on iteration 18
I am on iteration 19
I am on iteration 20
I am on iteration 21
I am on iteration 22
I am on iteration 23
I am on iteration 24
I am on iteration 25
[(10, 0.99911904132953666), (11, 0.999208408152783), (12, 0.9993210112590043), (13, 0.99599989926942012), (14, 0.97433351142380575), (15, 0.94387941285924981), (16, 0.95082815284679245), (17, 0.94344465258845644), (18, 0.81203662832850687), (19, 0.82599630426921467), (20, 0.62858428161894353), (21, 0.53134571854475965), (22, 0.60284728842384017), (23, 0.58005149559561919), (24, 0.58976373269590099), (25, 0.5598676947203759)]


In [73]:
news = pd.DataFrame(newsgroups['target'], columns=['labels'])

In [74]:
def text_labels(value):
    return newsgroups['target_names'][value]

print text_labels(16)

talk.politics.guns


In [75]:
news['text_labels'] = news['labels'].apply(text_labels)

In [76]:
final_model = KMeans(n_clusters=20)
scaled = StandardScaler().fit_transform(tsvd_all)
final_model.fit(tsvd_all)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=20, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [77]:
news = news.join(pd.DataFrame(final_model.labels_, columns=['predicted_group']))

In [78]:
news.groupby('predicted_group')['text_labels'].value_counts()

predicted_group  text_labels             
0                rec.motorcycles             545
                 sci.electronics             525
                 comp.graphics               513
                 sci.med                     511
                 comp.sys.mac.hardware       508
                 comp.windows.x              507
                 rec.autos                   507
                 misc.forsale                506
                 comp.sys.ibm.pc.hardware    505
                 rec.sport.baseball          495
                 rec.sport.hockey            479
                 sci.space                   474
                 comp.os.ms-windows.misc     466
                 sci.crypt                   463
                 soc.religion.christian      439
                 talk.politics.mideast       407
                 talk.politics.guns          401
                 alt.atheism                 393
                 talk.politics.misc          334
                 talk.relig

In [79]:
news['predicted_group'].value_counts()

0     9275
19    1867
18      99
14      36
13      17
16       4
12       3
2        1
8        1
1        1
9        1
17       1
15       1
10       1
7        1
11       1
4        1
5        1
6        1
3        1
Name: predicted_group, dtype: int64

In [ ]:
lets go back and strip some extra word crap out

In [80]:
df['stripped_marks'] = df['text_lower'].str.replace('>', '')

In [82]:
df['stripped_headers'] = df['text_lower'].str.replace('from:', '')
df['stripped_headers'] = df['text_lower'].str.replace('subject:', '')
df['stripped_headers'] = df['text_lower'].str.replace('lines:', '')